In [1]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x,dim=1)

In [2]:
PATH = './mnist_net.pth'

In [8]:
import torch

model = Net()
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [6]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

batch_size = 128
download_root = 'test-data'
mnist_transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))])
test_dataset  = datasets.MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=False)

In [20]:
import numpy as np

def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

def test(model, loss_func, optimizer, test_loader):
    losses = []
    nums = []
    accs = []
    model.eval()
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            output = model(data)
            loss = loss_func(output, target)
            acc = accuracy(output,target)
            losses.append(loss)
            accs.append(acc)
            nums.append(len(data))
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        val_acc = np.sum(np.multiply(accs, nums)) / np.sum(nums)
        print('Loss: {:.6f}\tAccuracy: {}'.format(val_loss, val_acc*100.))

In [13]:
import torch.nn.functional as F

loss_func = F.nll_loss

In [10]:
from torch import optim

lr = 0.1
optimizer = optim.SGD(model.parameters(), lr=lr)

In [21]:
test(model, loss_func, optimizer, test_loader)

Loss: 0.053974	Accuracy: 98.29


In [22]:
x = torch.randn(batch_size, 1, 224, 224, requires_grad=True)

In [24]:
x.shape

torch.Size([128, 1, 224, 224])

In [29]:
test_loader.dataset.test_data.shape

torch.Size([10000, 28, 28])

In [30]:
import onnx

# Load the ONNX model
model_onnx = onnx.load("mnist_net.onnx")

In [31]:
test(model_onnx, loss_func, optimizer, test_loader)

AttributeError: eval